In [25]:
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict


class PicogenConfig(BaseSettings):
    api_url: str = Field(..., env="PICOGEN_API_URL")
    api_key: str = Field(..., env="PICOGEN_API_KEY")

    model_config = SettingsConfigDict(
        env_file=".env",
        env_file_encoding="utf-8",
        extra="ignore",
        env_prefix="PICOGEN_",
        env_nested_delimiter="_",
    )


config = PicogenConfig()

In [26]:
from pydantic import BaseModel


class PicogenJobGenerateResponse(BaseModel):
    id: str
    cost: float

class PicogenJobGetResponse(BaseModel):
    id: str
    status: str
    payload: dict
    result: str
    duration_ms: int
    created_at: int


In [27]:
from time import sleep
from typing import Optional
from requests import HTTPError, Response, Session


class PicogenService:
    def __init__(self, config: PicogenConfig = config):
        self.session = Session()
        self.session.headers.update({"API-Token": config.api_key})

    def job_generate(self, prompt: str, ratio: str = "1:1", seed: Optional[int] = None):
        try:
            payload = {
                "prompt": prompt,
                "ratio": ratio,
            }
            if seed:
                payload["seed"] = seed
            response: Response = self.session.post(f"{config.api_url}/job/generate", json=payload)
            response.raise_for_status()
            responses: list[PicogenJobGenerateResponse] = [PicogenJobGenerateResponse(**r) for r in response.json() if r]
            return responses
        except HTTPError as e:
            print(f"Error generating job: {e.response.json()}")
            raise e
        except Exception as e:
            print(f"Error generating job: {e}")
            raise e

    def job_get(self, job_id: str):
        for _ in range(10):
            try: 
                response: Response = self.session.get(f"{config.api_url}/job/get/{job_id}")
                response.raise_for_status()
                responses: list[PicogenJobGetResponse] = [PicogenJobGetResponse(**r) for r in response.json() if r]
                return responses
            except HTTPError as e:
                print(f"Error getting job {job_id}: {e.response.json()}")
                sleep(1)
            except Exception as e:
                print(f"Error getting job {job_id}: {e}")
                raise e
            
        raise Exception(f"Failed to get job {job_id} after 10 attempts")


service = PicogenService()


In [ ]:
message = """Generate a neon-lit cybernetic owl perched on a crumbling stone archway in a misty swamp.
Its feathers are made of overlapping circuit boards, eyes glow with holographic constellations, and mechanical talons drip molten gold into the water below.
Surround the scene with bioluminescent vines and floating hologram runes.
Render in 4K with moody, cinematic lighting."""
responses = service.job_generate(message)
generate_response = responses[0]
print(generate_response.id, generate_response.cost)


WvqkZH59Fpx 8.0


In [30]:
responses = service.job_get(generate_response.id)
get_response = responses[0]
print(get_response.id, get_response.status, get_response.result)


WvqkZH59Fpx completed https://image.picogen.io/202502/02/a/5/a57c3a39707a800a1340c0aed03ec6c5.png
